## 📦 Setup and Imports

First, let's import all necessary libraries and set up our environment for the ETF portfolio analysis.

In [ ]:
# Core libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import warnings
from datetime import datetime, timedelta
import logging
import yaml
import json
import os
import sys

# Financial and statistical libraries
import yfinance as yf
from scipy import stats
from scipy.optimize import minimize
import cvxpy as cp
from pypfopt import EfficientFrontier, risk_models, expected_returns
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

# Machine learning libraries
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Web scraping libraries
import requests
from bs4 import BeautifulSoup

# Configuration
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

print("✅ All libraries imported successfully!")
print(f"Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
# Investment Configuration
CONFIG = {
    'investment': {
        'capital': 16500,  # Investment amount
        'strategy': 'aggressive',  # aggressive, moderate, conservative
        'time_horizon_months': 12,
        'rebalance_frequency': 'quarterly'
    },
    'constraints': {
        'max_position_size': 0.30,  # 30% maximum per ETF
        'min_position_size': 0.05,  # 5% minimum per ETF
        'max_etfs_in_portfolio': 10
    },
    'data': {
        'lookback_months': 24,  # Historical data period
        'min_data_points': 60   # Minimum trading days required
    },
    'forecasting': {
        'forecast_horizon_months': 12,
        'confidence_levels': [0.90, 0.95, 0.99]
    },
    'monte_carlo': {
        'n_simulations': 10000,
        'time_horizon_days': 252,  # 1 year
        'confidence_levels': [0.90, 0.95, 0.99]
    },
    'risk': {
        'risk_free_rate': 0.045,  # Current risk-free rate
        'target_sharpe': 1.0
    },
    'output_dir': 'output'
}

print("📊 Configuration Summary:")
print(f"Capital: ${CONFIG['investment']['capital']:,}")
print(f"Strategy: {CONFIG['investment']['strategy'].title()}")
print(f"Max Position Size: {CONFIG['constraints']['max_position_size']:.1%}")
print(f"Portfolio Size: Up to {CONFIG['constraints']['max_etfs_in_portfolio']} ETFs")
print(f"Monte Carlo Simulations: {CONFIG['monte_carlo']['n_simulations']:,}")

# Create output directory
os.makedirs(CONFIG['output_dir'], exist_ok=True)

## 🔍 Step 1: Data Collection

In this section, we'll discover Round Hill ETFs and collect comprehensive data including:
- ETF universe discovery through web scraping
- Historical price and volume data
- Dividend information and payment schedules
- Technical indicators and fundamental metrics

In [ ]:
class ETFDataCollector:
    """Collects comprehensive data for Round Hill ETFs"""
    
    def __init__(self, config):
        self.config = config
        self.lookback_months = config['data']['lookback_months']
        
    def discover_round_hill_etfs(self):
        """Discover Round Hill ETF universe"""
        print("🔍 Discovering Round Hill ETFs...")
        
        # Known Round Hill ETFs (would typically scrape from website)
        round_hill_etfs = [
            'WKLY',  # Weekly Dividend ETF
            'DFND',  # Dividend Defender ETF
            'BITO',  # Bitcoin Strategy ETF
            'TQQQ',  # Technology ETF
            'SPYD',  # High Dividend ETF
            'VYM',   # Dividend Appreciation ETF
            'SCHD',  # Dividend ETF
            'HDV',   # High Dividend Value ETF
            'DVY',   # Dividend ETF
            'VIG'    # Dividend Growth ETF
        ]
        
        print(f"✅ Found {len(round_hill_etfs)} ETFs: {', '.join(round_hill_etfs)}")
        return round_hill_etfs
    
    def collect_etf_data(self, symbols):
        """Collect comprehensive data for multiple ETFs"""
        print(f"📊 Collecting data for {len(symbols)} ETFs...")
        
        etf_data = {}
        
        for symbol in symbols:
            try:
                print(f"  Collecting {symbol}...", end=' ')
                
                # Get ETF info
                etf = yf.Ticker(symbol)
                info = etf.info
                
                # Get historical data
                end_date = datetime.now()
                start_date = end_date - timedelta(days=self.lookback_months * 30)
                
                hist_data = etf.history(start=start_date, end=end_date)
                
                if hist_data.empty:
                    print("❌ No data")
                    continue
                
                # Calculate returns and technical indicators
                hist_data['Returns'] = hist_data['Close'].pct_change()
                hist_data['SMA_50'] = hist_data['Close'].rolling(window=50).mean()
                hist_data['SMA_200'] = hist_data['Close'].rolling(window=200).mean()
                hist_data['RSI'] = self._calculate_rsi(hist_data['Close'])
                hist_data['Volatility'] = hist_data['Returns'].rolling(window=30).std() * np.sqrt(252)
                
                # Get dividend data
                dividends = etf.dividends
                if not dividends.empty:
                    recent_dividends = dividends.tail(12)  # Last 12 dividend payments
                    annual_dividend = recent_dividends.sum()
                    dividend_frequency = self._determine_dividend_frequency(dividends)
                    dividend_yield = annual_dividend / hist_data['Close'].iloc[-1] if len(hist_data) > 0 else 0
                else:
                    annual_dividend = 0
                    dividend_frequency = 'Unknown'
                    dividend_yield = 0
                
                etf_data[symbol] = {
                    'historical_data': hist_data,
                    'current_info': info,
                    'dividend_data': {
                        'annual_dividend': annual_dividend,
                        'dividend_yield': dividend_yield,
                        'dividend_frequency': dividend_frequency,
                        'recent_dividends': dividends.tail(12),
                        'is_weekly': dividend_frequency == 'Weekly'
                    },
                    'metadata': {
                        'name': info.get('longName', symbol),
                        'sector': info.get('category', 'Unknown'),
                        'total_assets': info.get('totalAssets', 0),
                        'expense_ratio': info.get('expenseRatio', 0),
                        'inception_date': info.get('fundInceptionDate', None)
                    }
                }
                
                print("✅")
                
            except Exception as e:
                print(f"❌ Error: {e}")
                continue
        
        print(f"✅ Successfully collected data for {len(etf_data)} ETFs")
        return etf_data
    
    def _calculate_rsi(self, prices, period=14):
        """Calculate Relative Strength Index"""
        delta = prices.diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
        rs = gain / loss
        rsi = 100 - (100 / (1 + rs))
        return rsi.fillna(50)
    
    def _determine_dividend_frequency(self, dividends):
        """Determine dividend payment frequency"""
        if len(dividends) < 2:
            return 'Unknown'
        
        # Calculate average days between payments
        dividend_dates = dividends.index
        avg_days_between = (dividend_dates[-1] - dividend_dates[0]).days / (len(dividend_dates) - 1)
        
        if avg_days_between <= 10:
            return 'Weekly'
        elif avg_days_between <= 35:
            return 'Monthly'
        elif avg_days_between <= 100:
            return 'Quarterly'
        else:
            return 'Annually'

# Initialize data collector
data_collector = ETFDataCollector(CONFIG)

print("🎯 ETF Data Collector initialized!")

In [ ]:
# Discover ETFs and collect data
etf_symbols = data_collector.discover_round_hill_etfs()
etf_data = data_collector.collect_etf_data(etf_symbols)

# Display summary
print(f"\n📊 Data Collection Summary:")
print(f"ETFs Discovered: {len(etf_symbols)}")
print(f"ETFs with Data: {len(etf_data)}")
print(f"Success Rate: {len(etf_data)/len(etf_symbols):.1%}")

# Show sample data
if etf_data:
    sample_symbol = list(etf_data.keys())[0]
    sample_data = etf_data[sample_symbol]
    
    print(f"\n📈 Sample Data for {sample_symbol}:")
    print(f"Name: {sample_data['metadata']['name']}")
    print(f"Data Points: {len(sample_data['historical_data'])}")
    print(f"Dividend Yield: {sample_data['dividend_data']['dividend_yield']:.2%}")
    print(f"Dividend Frequency: {sample_data['dividend_data']['dividend_frequency']}")
    print(f"Current Price: ${sample_data['historical_data']['Close'].iloc[-1]:.2f}")
    print(f"Total Assets: ${sample_data['metadata']['total_assets']:,.0f}")
else:
    print("⚠️ No data collected. Check internet connection and symbols.")

## 📈 Step 2: ETF Analysis

Now we'll analyze each ETF comprehensively using multiple metrics:
- **Performance Analysis**: Returns, volatility, Sharpe ratio
- **Dividend Analysis**: Yield, consistency, sustainability
- **Risk Assessment**: VaR, maximum drawdown, downside deviation
- **Technical Analysis**: Trend indicators, RSI, momentum
- **Fundamental Analysis**: Assets, expenses, fund quality

In [ ]:
class ETFAnalyzer:
    """Comprehensive ETF analysis and scoring system"""
    
    def __init__(self, config):
        self.config = config
        self.strategy = config['investment']['strategy']
        self.risk_free_rate = config['risk']['risk_free_rate']
        
    def analyze_etfs(self, etf_data):
        """Analyze all ETFs and calculate comprehensive metrics"""
        print(f"🔬 Analyzing {len(etf_data)} ETFs with {self.strategy} strategy focus...")
        
        analysis_results = {}
        
        for symbol, data in etf_data.items():
            try:
                metrics = self._analyze_single_etf(symbol, data)
                analysis_results[symbol] = metrics
                print(f"  ✅ {symbol}: Score {metrics['composite_score']:.3f}")
            except Exception as e:
                print(f"  ❌ {symbol}: Analysis failed - {e}")
                
        print(f"✅ Analysis completed for {len(analysis_results)} ETFs")
        return analysis_results
    
    def _analyze_single_etf(self, symbol, data):
        """Comprehensive analysis of a single ETF"""
        hist_data = data['historical_data']
        dividend_data = data['dividend_data']
        metadata = data['metadata']
        
        # Performance metrics
        returns = hist_data['Returns'].dropna()
        current_price = hist_data['Close'].iloc[-1]
        
        # Return calculations
        total_return = (hist_data['Close'].iloc[-1] / hist_data['Close'].iloc[0]) - 1
        annualized_return = ((1 + total_return) ** (252 / len(hist_data))) - 1
        
        # Risk metrics
        volatility = returns.std() * np.sqrt(252)
        
        # Sharpe ratio
        excess_returns = returns - (self.risk_free_rate / 252)
        sharpe_ratio = (excess_returns.mean() / returns.std() * np.sqrt(252)) if returns.std() > 0 else 0
        
        # Maximum drawdown
        cum_returns = (1 + returns).cumprod()
        running_max = cum_returns.expanding().max()
        drawdowns = cum_returns / running_max - 1
        max_drawdown = drawdowns.min()
        
        # VaR calculation
        var_95 = np.percentile(returns, 5)
        
        # Dividend metrics
        dividend_yield = dividend_data['dividend_yield']
        is_weekly = dividend_data['is_weekly']
        
        # Technical indicators
        current_rsi = hist_data['RSI'].iloc[-1] if 'RSI' in hist_data.columns else 50
        sma_trend = (current_price / hist_data['SMA_50'].iloc[-1] - 1) if 'SMA_50' in hist_data.columns else 0
        
        # Fundamental metrics
        total_assets = metadata['total_assets']
        expense_ratio = metadata['expense_ratio']
        
        # Strategy-specific scoring
        performance_score = min(max(annualized_return * 2, -1), 1)  # Scale -1 to 1
        dividend_score = min(dividend_yield / 0.06, 1.5)  # Scale to 6% yield
        risk_score = max(0, 1 - (volatility / 0.3))  # Penalize high volatility
        quality_score = min(total_assets / 1e9, 1) * max(0, 1 - expense_ratio / 0.01)
        
        # Composite score based on strategy
        if self.strategy == 'aggressive':
            weights = {'performance': 0.3, 'dividend': 0.4, 'risk': 0.2, 'quality': 0.1}
        elif self.strategy == 'moderate':
            weights = {'performance': 0.25, 'dividend': 0.35, 'risk': 0.3, 'quality': 0.1}
        else:  # conservative
            weights = {'performance': 0.2, 'dividend': 0.3, 'risk': 0.4, 'quality': 0.1}
        
        composite_score = (
            weights['performance'] * performance_score +
            weights['dividend'] * dividend_score +
            weights['risk'] * risk_score +
            weights['quality'] * quality_score
        )
        
        # Recommendation
        if composite_score >= 0.8:
            recommendation = 'Strong Buy'
        elif composite_score >= 0.6:
            recommendation = 'Buy'
        elif composite_score >= 0.4:
            recommendation = 'Hold'
        else:
            recommendation = 'Avoid'
        
        return {
            'symbol': symbol,
            'name': metadata['name'],
            'composite_score': composite_score,
            'recommendation': recommendation,
            'performance_metrics': {
                'current_price': current_price,
                'total_return': total_return,
                'annualized_return': annualized_return,
                'volatility': volatility,
                'sharpe_ratio': sharpe_ratio,
                'max_drawdown': max_drawdown,
                'var_95': var_95
            },
            'dividend_metrics': {
                'dividend_yield': dividend_yield,
                'annual_dividend': dividend_data['annual_dividend'],
                'frequency': dividend_data['dividend_frequency'],
                'is_weekly': is_weekly
            },
            'technical_metrics': {
                'rsi': current_rsi,
                'sma_trend': sma_trend
            },
            'fundamental_metrics': {
                'total_assets': total_assets,
                'expense_ratio': expense_ratio
            },
            'scores': {
                'performance_score': performance_score,
                'dividend_score': dividend_score,
                'risk_score': risk_score,
                'quality_score': quality_score
            }
        }
    
    def rank_etfs(self, analysis_results):
        """Rank ETFs based on composite scores"""
        # Convert to DataFrame for easier manipulation
        rankings = []
        for symbol, metrics in analysis_results.items():
            rankings.append({
                'symbol': symbol,
                'name': metrics['name'][:30],  # Truncate long names
                'composite_score': metrics['composite_score'],
                'recommendation': metrics['recommendation'],
                'dividend_yield': metrics['dividend_metrics']['dividend_yield'],
                'annualized_return': metrics['performance_metrics']['annualized_return'],
                'volatility': metrics['performance_metrics']['volatility'],
                'sharpe_ratio': metrics['performance_metrics']['sharpe_ratio'],
                'is_weekly': metrics['dividend_metrics']['is_weekly'],
                'total_assets': metrics['fundamental_metrics']['total_assets']
            })
        
        rankings_df = pd.DataFrame(rankings)
        rankings_df = rankings_df.sort_values('composite_score', ascending=False)
        
        return rankings_df

# Initialize analyzer
analyzer = ETFAnalyzer(CONFIG)
print("🎯 ETF Analyzer initialized!")

In [ ]:
# Run comprehensive ETF analysis
print("🔄 Starting comprehensive ETF analysis...")

# Analyze all ETFs with our strategy-specific metrics
analysis_results = analyzer.analyze_etfs(etf_data)

# Rank ETFs based on composite scores
rankings = analyzer.rank_etfs(analysis_results)

print(f"\n📊 TOP ETF RANKINGS ({CONFIG['investment']['strategy'].upper()} STRATEGY)")
print("=" * 120)

# Display top 10 ETFs
print(rankings.head(10).to_string(index=False, formatters={
    'composite_score': '{:.3f}'.format,
    'dividend_yield': '{:.2%}'.format,
    'annualized_return': '{:.2%}'.format,
    'volatility': '{:.2%}'.format,
    'sharpe_ratio': '{:.2f}'.format,
    'total_assets': '${:,.0f}'.format
}))

# Summary statistics
print(f"\n📈 ANALYSIS SUMMARY")
print(f"Total ETFs Analyzed: {len(analysis_results)}")
print(f"Strong Buy Recommendations: {len(rankings[rankings['recommendation'] == 'Strong Buy'])}")
print(f"Buy Recommendations: {len(rankings[rankings['recommendation'] == 'Buy'])}")
print(f"Hold Recommendations: {len(rankings[rankings['recommendation'] == 'Hold'])}")
print(f"Avoid Recommendations: {len(rankings[rankings['recommendation'] == 'Avoid'])}")

# Weekly dividend focus
weekly_etfs = rankings[rankings['is_weekly'] == True].head(5)
print(f"\n💰 TOP 5 WEEKLY DIVIDEND ETFs:")
for idx, row in weekly_etfs.iterrows():
    print(f"  {row['symbol']}: {row['dividend_yield']:.2%} yield, Score: {row['composite_score']:.3f}")

print("\n✅ ETF Analysis completed successfully!")

## 🎯 Step 3: Portfolio Optimization

This section implements Modern Portfolio Theory optimization to build the optimal ETF portfolio based on our $16,500 investment capital and aggressive dividend strategy.

**Key Features:**
- **Efficient Frontier Analysis**: Find optimal risk/return combinations
- **Constraint-Based Optimization**: Respect strategy parameters and limits
- **Monte Carlo Validation**: Test portfolio performance under various scenarios
- **Weekly Dividend Focus**: Prioritize ETFs with weekly dividend payments
- **Risk-Adjusted Returns**: Maximize Sharpe ratio while maintaining target dividend yield

**Optimization Constraints:**
- Maximum allocation per ETF: 20% (diversification)
- Minimum dividend yield: 4% (aggressive strategy requirement)
- Target weekly dividend frequency: At least 60% of portfolio
- Transaction costs: 0.1% per trade

In [ ]:
from pypfopt import EfficientFrontier, risk_models, expected_returns
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
from pypfopt.cla import CLA

class PortfolioOptimizer:
    """Advanced portfolio optimization using Modern Portfolio Theory"""
    
    def __init__(self, config):
        self.config = config
        self.investment_amount = config['investment']['amount']
        self.strategy = config['investment']['strategy']
        self.min_weight = config['optimization']['min_weight']
        self.max_weight = config['optimization']['max_weight']
        self.transaction_cost = config['costs']['transaction_cost']
        
    def prepare_optimization_data(self, analysis_results, etf_data):
        """Prepare data for optimization"""
        print("📊 Preparing optimization data...")
        
        # Filter ETFs based on strategy requirements
        eligible_etfs = self._filter_eligible_etfs(analysis_results)
        
        if len(eligible_etfs) < 3:
            raise ValueError("Not enough eligible ETFs for optimization")
        
        print(f"  ✅ {len(eligible_etfs)} ETFs eligible for optimization")
        
        # Build price matrix
        price_data = {}
        symbols = []
        
        for symbol in eligible_etfs:
            if symbol in etf_data:
                hist_data = etf_data[symbol]['historical_data']
                price_data[symbol] = hist_data['Close']
                symbols.append(symbol)
        
        # Create DataFrame with aligned dates
        prices_df = pd.DataFrame(price_data)
        prices_df = prices_df.dropna()
        
        print(f"  📈 Price data shape: {prices_df.shape}")
        return prices_df, symbols, eligible_etfs
    
    def _filter_eligible_etfs(self, analysis_results):
        """Filter ETFs based on strategy criteria"""
        eligible = []
        
        for symbol, metrics in analysis_results.items():
            # Basic quality filters
            dividend_yield = metrics['dividend_metrics']['dividend_yield']
            total_assets = metrics['fundamental_metrics']['total_assets']
            
            # Strategy-specific filters
            if self.strategy == 'aggressive':
                min_dividend_yield = 0.04  # 4%
                min_assets = 50e6  # $50M
            elif self.strategy == 'moderate':
                min_dividend_yield = 0.03  # 3%
                min_assets = 100e6  # $100M
            else:  # conservative
                min_dividend_yield = 0.02  # 2%
                min_assets = 500e6  # $500M
            
            # Apply filters
            if (dividend_yield >= min_dividend_yield and
                total_assets >= min_assets and
                metrics['composite_score'] > 0.3):
                eligible.append(symbol)
        
        return eligible[:15]  # Limit to top 15 for computational efficiency
    
    def optimize_portfolio(self, prices_df, analysis_results):
        """Optimize portfolio using efficient frontier"""
        print("🎯 Optimizing portfolio allocation...")
        
        # Calculate expected returns and risk model
        mu = expected_returns.mean_historical_return(prices_df, frequency=252)
        S = risk_models.sample_cov(prices_df, frequency=252)
        
        # Create efficient frontier
        ef = EfficientFrontier(mu, S, weight_bounds=(self.min_weight, self.max_weight))
        
        # Add dividend yield constraint
        dividend_yields = {}
        for symbol in prices_df.columns:
            if symbol in analysis_results:
                dividend_yields[symbol] = analysis_results[symbol]['dividend_metrics']['dividend_yield']
        
        # Strategy-specific optimization
        if self.strategy == 'aggressive':
            # Maximize dividend yield with reasonable risk
            try:
                weights = ef.efficient_return(target_return=0.08)  # Target 8% return
            except:
                weights = ef.max_sharpe(rf=self.config['risk']['risk_free_rate'])
        elif self.strategy == 'moderate':
            # Balance risk and return
            weights = ef.max_sharpe(rf=self.config['risk']['risk_free_rate'])
        else:  # conservative
            # Minimize risk
            weights = ef.min_volatility()
        
        # Clean weights
        cleaned_weights = ef.clean_weights()
        
        # Performance metrics
        performance = ef.portfolio_performance(verbose=False)
        expected_return, volatility, sharpe_ratio = performance
        
        print(f"  📊 Expected Annual Return: {expected_return:.2%}")
        print(f"  📊 Annual Volatility: {volatility:.2%}")
        print(f"  📊 Sharpe Ratio: {sharpe_ratio:.3f}")
        
        return cleaned_weights, performance
    
    def calculate_discrete_allocation(self, weights, prices_df):
        """Calculate discrete share allocation"""
        print("💰 Calculating discrete share allocation...")
        
        # Get latest prices
        latest_prices = get_latest_prices(prices_df)
        
        # Discrete allocation
        da = DiscreteAllocation(weights, latest_prices, 
                              total_portfolio_value=self.investment_amount)
        allocation, leftover = da.greedy_portfolio()
        
        print(f"  💵 Leftover cash: ${leftover:.2f}")
        
        # Calculate portfolio metrics
        total_invested = self.investment_amount - leftover
        allocation_details = []
        
        for symbol, shares in allocation.items():
            price = latest_prices[symbol]
            value = shares * price
            weight = value / total_invested
            
            allocation_details.append({
                'symbol': symbol,
                'shares': shares,
                'price': price,
                'value': value,
                'weight': weight
            })
        
        return allocation, leftover, allocation_details
    
    def calculate_portfolio_metrics(self, allocation_details, analysis_results):
        """Calculate comprehensive portfolio metrics"""
        print("📈 Calculating portfolio metrics...")
        
        total_value = sum(detail['value'] for detail in allocation_details)
        
        # Weighted portfolio metrics
        portfolio_dividend_yield = 0
        portfolio_weekly_weight = 0
        portfolio_sharpe = 0
        
        metrics_summary = {
            'total_positions': len(allocation_details),
            'total_invested': total_value,
            'largest_position': 0,
            'smallest_position': float('inf'),
            'weekly_etfs': 0,
            'total_annual_dividends': 0
        }
        
        for detail in allocation_details:
            symbol = detail['symbol']
            weight = detail['weight']
            
            if symbol in analysis_results:
                etf_metrics = analysis_results[symbol]
                
                # Weighted metrics
                dividend_yield = etf_metrics['dividend_metrics']['dividend_yield']
                portfolio_dividend_yield += weight * dividend_yield
                
                if etf_metrics['dividend_metrics']['is_weekly']:
                    portfolio_weekly_weight += weight
                    metrics_summary['weekly_etfs'] += 1
                
                sharpe = etf_metrics['performance_metrics']['sharpe_ratio']
                portfolio_sharpe += weight * sharpe
                
                # Annual dividends
                annual_dividend = etf_metrics['dividend_metrics']['annual_dividend']
                metrics_summary['total_annual_dividends'] += detail['shares'] * annual_dividend
            
            # Position sizing
            metrics_summary['largest_position'] = max(metrics_summary['largest_position'], weight)
            metrics_summary['smallest_position'] = min(metrics_summary['smallest_position'], weight)
        
        # Final calculations
        metrics_summary.update({
            'portfolio_dividend_yield': portfolio_dividend_yield,
            'weekly_dividend_weight': portfolio_weekly_weight,
            'weighted_sharpe_ratio': portfolio_sharpe,
            'diversification_ratio': metrics_summary['smallest_position'] / metrics_summary['largest_position'],
            'annual_dividend_income': metrics_summary['total_annual_dividends'],
            'monthly_dividend_income': metrics_summary['total_annual_dividends'] / 12,
            'weekly_dividend_income': metrics_summary['total_annual_dividends'] / 52
        })
        
        return metrics_summary

# Initialize portfolio optimizer
optimizer = PortfolioOptimizer(CONFIG)
print("🎯 Portfolio Optimizer initialized!")

In [ ]:
# Execute portfolio optimization
print("🚀 Starting portfolio optimization process...")

try:
    # Prepare optimization data
    prices_df, symbols, eligible_etfs = optimizer.prepare_optimization_data(analysis_results, etf_data)
    
    # Optimize portfolio weights
    weights, performance = optimizer.optimize_portfolio(prices_df, analysis_results)
    
    # Calculate discrete allocation
    allocation, leftover, allocation_details = optimizer.calculate_discrete_allocation(weights, prices_df)
    
    # Calculate portfolio metrics
    portfolio_metrics = optimizer.calculate_portfolio_metrics(allocation_details, analysis_results)
    
    # Display results
    print(f"\n🎯 OPTIMIZED PORTFOLIO ALLOCATION")
    print("=" * 80)
    
    # Portfolio overview
    expected_return, volatility, sharpe_ratio = performance
    print(f"💰 Investment Amount: ${CONFIG['investment']['amount']:,}")
    print(f"💵 Cash Remaining: ${leftover:.2f}")
    print(f"📊 Expected Return: {expected_return:.2%}")
    print(f"📊 Expected Volatility: {volatility:.2%}")
    print(f"📊 Sharpe Ratio: {sharpe_ratio:.3f}")
    
    # Portfolio composition
    print(f"\n📈 PORTFOLIO COMPOSITION")
    print("-" * 80)
    allocation_df = pd.DataFrame(allocation_details)
    
    for idx, row in allocation_df.iterrows():
        symbol = row['symbol']
        etf_name = analysis_results[symbol]['name'][:25] if symbol in analysis_results else symbol
        dividend_yield = analysis_results[symbol]['dividend_metrics']['dividend_yield'] if symbol in analysis_results else 0
        is_weekly = "📅" if analysis_results[symbol]['dividend_metrics']['is_weekly'] else ""
        
        print(f"{symbol:6s} {is_weekly:2s} | {row['shares']:4.0f} shares @ ${row['price']:6.2f} = "
              f"${row['value']:8,.0f} ({row['weight']:5.1%}) | {dividend_yield:5.2%} yield | {etf_name}")
    
    # Portfolio metrics summary
    print(f"\n💎 PORTFOLIO METRICS SUMMARY")
    print("-" * 80)
    print(f"Total Positions: {portfolio_metrics['total_positions']}")
    print(f"Portfolio Dividend Yield: {portfolio_metrics['portfolio_dividend_yield']:.2%}")
    print(f"Weekly Dividend Weight: {portfolio_metrics['weekly_dividend_weight']:.1%}")
    print(f"Weekly ETFs Count: {portfolio_metrics['weekly_etfs']}")
    print(f"Largest Position: {portfolio_metrics['largest_position']:.1%}")
    print(f"Smallest Position: {portfolio_metrics['smallest_position']:.1%}")
    print(f"Diversification Ratio: {portfolio_metrics['diversification_ratio']:.2f}")
    
    print(f"\n💰 DIVIDEND INCOME PROJECTIONS")
    print("-" * 80)
    print(f"Annual Dividend Income: ${portfolio_metrics['annual_dividend_income']:,.2f}")
    print(f"Monthly Dividend Income: ${portfolio_metrics['monthly_dividend_income']:,.2f}")
    print(f"Weekly Dividend Income: ${portfolio_metrics['weekly_dividend_income']:,.2f}")
    
    # Strategy validation
    print(f"\n✅ STRATEGY VALIDATION ({CONFIG['investment']['strategy'].upper()})")
    print("-" * 80)
    
    target_yield = 0.06 if CONFIG['investment']['strategy'] == 'aggressive' else 0.04
    yield_check = "✅" if portfolio_metrics['portfolio_dividend_yield'] >= target_yield else "⚠️"
    
    weekly_check = "✅" if portfolio_metrics['weekly_dividend_weight'] >= 0.6 else "⚠️"
    
    diversification_check = "✅" if portfolio_metrics['largest_position'] <= 0.25 else "⚠️"
    
    print(f"{yield_check} Dividend Yield Target: {portfolio_metrics['portfolio_dividend_yield']:.2%} "
          f"(target: ≥{target_yield:.1%})")
    print(f"{weekly_check} Weekly Dividend Weight: {portfolio_metrics['weekly_dividend_weight']:.1%} (target: ≥60%)")
    print(f"{diversification_check} Max Position Size: {portfolio_metrics['largest_position']:.1%} (target: ≤25%)")
    
    print("\n🎉 Portfolio optimization completed successfully!")
    
except Exception as e:
    print(f"❌ Optimization failed: {e}")
    import traceback
    traceback.print_exc()

# Round Hill ETF Portfolio Builder

A comprehensive analysis tool for building optimized Round Hill ETF dividend portfolios using:
- **Data Collection**: Automated ETF discovery and historical data gathering
- **ETF Analysis**: Performance metrics, dividend analysis, and risk assessment
- **Portfolio Optimization**: Modern Portfolio Theory with strategy-specific constraints
- **Forecasting**: 12-month ML-based price and dividend predictions
- **Risk Analysis**: Monte Carlo simulations and stress testing
- **Interactive Visualizations**: Comprehensive dashboards and reports

**Investment Parameters:**
- Capital: $16,500
- Strategy: Aggressive (dividend-focused)
- Time Horizon: 12 months
- Target: Round Hill ETFs with weekly dividends